In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

folder_path = '/content/drive'

# Mudar o diretório de trabalho para essa pasta
try:
    os.chdir(folder_path)
    print(f"Diretório de trabalho atual alterado para: {os.getcwd()}")
    print("Listando conteúdo do diretório para verificação:")
    !ls -F
except FileNotFoundError:
    print(f"ERRO: O caminho '{folder_path}' não foi encontrado. Por favor, verifique e corrija o caminho.")
except Exception as e:
    print(f"Ocorreu um erro ao mudar para o diretório '{folder_path}': {e}")


Diretório de trabalho atual alterado para: /content/drive/My Drive/teste_marcacao/revisar/resultados
Listando conteúdo do diretório para verificação:
 conteudo_lote_0406_filtrado.xlsx   conteudo_lote_21_filtrado.xlsx
 conteudo_lote_0710_filtrado.xlsx   conteudo_lote_22_filtrado.xlsx
 conteudo_lote_1114_filtrado.xlsx   conteudo_lote_23_filtrado.xlsx
 conteudo_lote_15_filtrado.xlsx     conteudo_lote_24_filtrado.xlsx
 conteudo_lote_16_filtrado.xlsx     conteudo_lote_25_filtrado.xlsx
 conteudo_lote_17_filtrado.xlsx    'Cópia de unir_arquivos.ipynb'
 conteudo_lote_18_filtrado.xlsx     filtrados/
 conteudo_lote_19_filtrado.xlsx     unir_arquivos.ipynb
 conteudo_lote_20_filtrado.xlsx


In [ ]:
import pandas as pd
import glob
import os
import re


def sanitize_excel_string(text):
    if isinstance(text, str):
                clean_text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f]', '', text)
        return clean_text
    return text

# Define as colunas a serem excluídas
colunas_para_excluir = ['var1', 'var2', 'var3']

# Identifica todos os arquivos 'conteudo*.xlsx' no diretório de trabalho atual
# Exclui arquivos que contêm 'unificado' ou 'limpo' no nome base
all_found_files_xlsx = glob.glob('conteudo*.xlsx')
files_to_clean_full_paths = [
    f for f in all_found_files_xlsx
    if 'unificado' not in os.path.basename(f).lower() and 'limpo' not in os.path.basename(f).lower()
]

print("Arquivos .xlsx identificados para limpeza individual:")
if files_to_clean_full_paths:
    for f_path in files_to_clean_full_paths:
        print(f" - {f_path}")
    print(f"\nTotal de {len(files_to_clean_full_paths)} arquivos .xlsx prontos para limpeza individual.\n")

    for file_path in files_to_clean_full_paths:
        print(f"\nProcessando arquivo: {file_path}")
        try:
            df = pd.read_excel(file_path)
            read_type = 'Excel'
        except Exception as excel_e:
            print(f"  - Não foi possível ler como Excel, tentando ler como CSV: {excel_e}")
            try:
                df = pd.read_csv(file_path)
                read_type = 'CSV'
            except Exception as csv_e:
                print(f"  - Não foi possível ler como CSV. Pulando arquivo: {csv_e}")
                continue

        original_rows = len(df)
        original_columns = len(df.columns)
        print(f"  - Arquivo lido com sucesso como {read_type} com {original_rows} linhas e {original_columns} colunas.")

        for col in df.select_dtypes(include=['object']).columns:
            df[col] = df[col].apply(sanitize_excel_string)
        print("  - Colunas de string saneadas para compatibilidade com Excel.")

        if 'marcacao_X' in df.columns:
            rows_before_filter = len(df)
            df = df[df['marcacao_X'].str.lower() != 'x']
            rows_after_filter = len(df)
            print(f"  - Excluídas {rows_before_filter - rows_after_filter} linhas com 'x' na coluna 'marcacao_X'.")
        else:
            print("  - Coluna 'marcacao_X' não encontrada. Nenhuma linha excluída com base nela.")

        colunas_existentes_para_excluir = [col for col in colunas_para_excluir if col in df.columns]
        if colunas_existentes_para_excluir:
            df = df.drop(columns=colunas_existentes_para_excluir)
            print(f"  - Colunas excluídas: {', '.join(colunas_existentes_para_excluir)}.")
        else:
            print("  - Nenhuma das colunas especificadas para exclusão foi encontrada.")

        base_name = os.path.basename(file_path)
        file_name_without_ext = os.path.splitext(base_name)[0]
        new_file_name = file_name_without_ext + '_limpo.xlsx'

        output_dir = os.path.dirname(file_path) if os.path.dirname(file_path) else '.' # Usar o diretório atual se não houver um caminho explícito
        output_path = os.path.join(output_dir, new_file_name)
        try:
            df.to_excel(output_path, index=False)
            print(f"  - Novo arquivo '{new_file_name}' salvo com sucesso em '{output_dir}'.")
            print(f"  - Arquivo limpo: {len(df)} linhas, {len(df.columns)} colunas.")
        except Exception as save_e:
            print(f"  - Erro ao salvar o arquivo '{new_file_name}': {save_e}. Pulando o salvamento para este arquivo.")

else:
    print("Nenhum arquivo 'conteudo*.xlsx' (não unificado ou limpo) encontrado no diretório atual.")
    print("Por favor, verifique se os arquivos corretos estão na pasta e não são unificados/limpos.")

print("\nTodos os processamentos foram concluídos.")


Arquivos .xlsx identificados para limpeza individual:
 - conteudo_lote_1114_filtrado.xlsx
 - conteudo_lote_0710_filtrado.xlsx
 - conteudo_lote_0406_filtrado.xlsx
 - conteudo_lote_25_filtrado.xlsx
 - conteudo_lote_24_filtrado.xlsx
 - conteudo_lote_23_filtrado.xlsx
 - conteudo_lote_22_filtrado.xlsx
 - conteudo_lote_21_filtrado.xlsx
 - conteudo_lote_20_filtrado.xlsx
 - conteudo_lote_19_filtrado.xlsx
 - conteudo_lote_18_filtrado.xlsx
 - conteudo_lote_17_filtrado.xlsx
 - conteudo_lote_16_filtrado.xlsx
 - conteudo_lote_15_filtrado.xlsx

Total de 14 arquivos .xlsx prontos para limpeza individual.


Processando arquivo: conteudo_lote_1114_filtrado.xlsx
  - Arquivo lido com sucesso como Excel com 4593 linhas e 5 colunas.
  - Colunas de string saneadas para compatibilidade com Excel.
  - Excluídas 4106 linhas com 'x' na coluna 'marcacao_X'.
  - Colunas excluídas: prob_X, marcacao_X, status_revisao.
  - Novo arquivo 'conteudo_lote_1114_filtrado_limpo.xlsx' salvo com sucesso em '.'.
  - Arquivo li